# 3 Задание. Практикум по программированию. Барилович Елизавета Игоревна ТРПО24-2
**Базовая часть:**
На базе модулей: csv, pickle и прямой работы с файлами реализовать следующий базовый функционал:
1.	функций *load_table*, *save_table* по загрузке/сохранению табличных данных во внутреннее представление модуля/из внутреннего представления модуля:
  
    o	файла формата csv (отдельный модуль с load_table, save_table в рамках общего пакета)
    o	файла формата pickle (отдельный модуль с load_table, save_table в рамках общего пакета), модуль использует структуру данных для представления таблицу, удобную автору работы.
    o	текстового файла (только функция save_table сохраняющая в текстовом файле представление таблицы, аналогичное выводу на печать с помощью функции print_table()).
Примечание: внутреннее представление может базироваться на словаре, где по разным ключам хранятся ключевые «атрибуты» таблицы, а значения таблицы хранятся в виде вложенных списков.

2.	модуля с базовыми операциями над таблицами:
o	*get_rows_by_number* – получение таблицы из одной строки или из строк из интервала по номеру строки. Функция либо копирует исходные данные, либо создает новое представление таблицы, работающее с исходным набором данных (copy_table=False), таким образом изменения, внесенные через это представления будут наблюдаться и в исходной таблице.


o	*get_rows_by_index*– получение новой таблицы из одной строки или из строк со значениями в первом столбце, совпадающими с переданными аргументами val1, … , valN. Функция либо копирует исходные данные, либо создает новое представление таблицы, работающее с исходным набором данных (copy_table=False), таким образом изменения, внесенные через это представления будут наблюдаться и в исходной таблице.

o	*get_column_types* – получение словаря вида столбец:тип_значений. Тип значения: int, float, bool, str (по умолчанию для всех столбцов). Параметр by_number определяет вид значения столбец – целочисленный индекс столбца или его строковое представление.

o	*set_column_types* – задание словаря вида столбец:тип_значений. Тип значения: int, float, bool, str (по умолчанию для всех столбцов). Параметр by_number определяет вид значения столбец – целочисленный индекс столбца или его строковое представление.

o	*get_values* – получение списка значений (типизированных согласно типу столбца) таблицы из столбца либо по номеру столбца (целое число, значение по умолчанию 0, либо по имени столбца)

o	*get_value* – аналог get_values(column=0) для представления таблицы с одной строкой, возвращает не список, а одно значение (типизированное согласно типу столбца).

o	*set_values* – задание списка значений values для столбца таблицы (типизированных согласно типу столбца) либо по номеру столбца (целое число, значение по умолчанию 0, либо по имени столбца).

o	*set_value* – аналог set_values(value, column=0) для представления таблицы с одной строкой, устанавливает не список значений, а одно значение (типизированное согласно типу столбца).

o	*print_table* – вывод таблицы на печать.
Для каждой функции должно быть реализована генерация не менее одного вида исключительных ситуаций.

**Дополнительные задания:**

Баллы - 2

4)	Реализовать автоматическое определение типа столбцов по хранящимся в таблице значениям. Оформить как отдельную функцию и встроить этот функционал как опцию работы функции load_table. 

Баллы - 2

5)	Реализовать поддержку дополнительного типа значений «дата и время» на основе модуля datetime.



In [1]:
import csv
import copy
import pickle
from datetime import datetime

In [2]:
def load_table(fl,autotype = False):
    """
    Loads a table from a file.
    This module provides a function to load data from various file formats
    (pickle, csv, txt) into a dictionary.  It also offers basic
    automatic type conversion(bool,ty).
    Loads a table from a file.

    Args:
        fl: Path to the file.  Supports '.pickle', '.csv', and '.txt' extensions.
        autotype: Boolean flag. If True, attempts to automatically convert
            column data types to datetime, boolean, integer, or float. Defaults to False.

    Returns:
        A dictionary where keys are column headers and values are lists of column data.
        Returns 'ошибка ввода!' if an error occurs during file processing.


    Raises:
         All exceptions are caught and a generic error message is returned.

    Example Usage:
        >>> table = load_table('data.csv', autotype=True)
        >>> print(table)
        {'column1': [1, 2, 3], 'column2': ['a', 'b', 'c']}

    """
    try:
        if fl.endswith('.pickle'):
            with open(fl, 'rb') as file:
                dic = pickle.load(file)
        with open(fl) as f:
            if fl.endswith('.csv'):
                f_csv = csv.reader(f, delimiter=';')
                header = next(f_csv)
                rows = [r for r in f_csv]
                coll = []
                for col in range(len(header)):
                    lst1 = []
                    for ln in range(len(rows)):
                        lst1.append(rows[ln][col])
                    coll.append(lst1)
                dic = {header[num]: coll[num] for num in range(len(header))}
            if fl.endswith('.txt'):
                header = f.readline().rstrip().split(' ')
                rows = [ft.rstrip().split(' ') for ft in f.readlines()]
                coll = []
                for col in range(len(header)):
                    lst1 = []
                    for ln in range(len(rows)):
                        lst1.append(rows[ln][col])
                    coll.append(lst1)
                dic = {header[num]: coll[num] for num in range(len(header))}
            if autotype == True:
                keyt = list(dic.keys())
                for ky in keyt:
                    lst = dic[ky]
                    try:
                        for el in range(len(lst)):
                            str_to_dt = lst[el]
                            lst[el] = datetime.strptime(str_to_dt, '%d.%m.%Y %H:%M')
                        dic[ky] = lst
                    except:
                        try:
                            if lst[0] == 'True' or lst[0] == 'False':
                                dic[ky] = list(map(bool, lst))
    
                        except: pass
                        try:
                            if str(int(lst[0])) == lst[0]:
                                lst = list(map(int, lst))
                                dic[ky] = lst
                        except: pass
                        try:
                            if str(float(lst[0])) == lst[0]:
                                lst = list(map(float, lst))
    
                                dic[ky] = lst
                        except: pass
            return dic
    except: return 'ошибка ввода!'

In [3]:
def save_table(dic,name,mode = 'csv'):
    """
    Saves a dictionary to a CSV, pickle, or TXT file.
    
    Args:
        dic: The dictionary to save (dict).  Values must be lists of equal length.
        name: The base filename (str). The file extension will be added based on the mode.
        mode: The file format ('csv', 'pickle', or 'txt'). Defaults to 'csv'.
    
    Returns:
        'таблица создана' if the table was saved successfully, or an error
        message 'ошибка ввода!' if an error occurred.
    
    Raises:
        All exceptions are caught and a generic error message is returned.
    
    Example Usage:
      >>> save_table({'col1': [1, 2, 3], 'col2': [4, 5, 6]}, 'my_data')  # Saves as CSV
      таблица создана
    
    """
    try:
        if mode == 'csv':
            key = list(dic.keys())
            val = list(dic.values())
            coll = []
            for col in range(len(key)):
                lst1 = []
                for ln in range(len(val)):
                    lst1.append(val[ln][col])
                coll.append(lst1)
            with open(name+'.csv', 'w+', newline='') as f:
                writer = csv.writer(f, delimiter=";")
                res = []
                res.append(key)
                for ln in range(len(key)):
                    line = []
                    for cl in range(len(val)):
                        line.append(str(val[cl][ln]))
                    res.append(line)
                writer.writerows(res)
        if mode == 'pickle':
            with open(name + '.pickle', 'wb') as file:
                pickle.dump(dic, file)
        if mode == 'txt':
            with open(name+'.txt', 'w+') as file:
                key = dic.keys()
                val = list(dic.values())
                file.write(' '.join([str(x) + ' ' for x in key])+'\n')
                for ln in range(len(key)):
                    line = []
                    for cl in range(len(val)):
                        line.append(val[cl][ln])
                    file.write(''.join([str(x) + ' ' for x in line])+'\n')
        return 'таблица создана'

    except:return 'ошибка ввода!'


In [4]:
tab = load_table('table.csv',True)
print(tab,'\n')
print(save_table(tab,'output','csv'))

print('txt')
tabtxt = load_table('tabletext.txt')
print(tabtxt,'\n')
print(save_table(tabtxt,'tabtxtnew','txt'))

print('pickle')
with open('tablepic.pickle', 'wb') as file:
    pickle.dump(tab, file)
tabpickle = load_table('tablepic.pickle')
print(tabpickle,'\n')
print(save_table(tabpickle,'tabpicklenew','pickle'))

{'name': ['Yana', 'Maria', 'Maxim', 'Vladimir', 'Peter'], 'surname': ['Dolgih', 'Petrova', 'Ivanov', 'Sidorov', 'Gavrilov'], 'age': [20, 30, 22, 52, 15], 'have_pets': [True, True, True, True, True], 'reg_date': [datetime.datetime(2024, 11, 23, 12, 30), datetime.datetime(2024, 11, 24, 12, 40), datetime.datetime(2024, 11, 25, 12, 32), datetime.datetime(2024, 11, 26, 12, 50), datetime.datetime(2024, 11, 27, 12, 20)]} 

таблица создана
txt
{'Name': ['Yana', 'Maria', 'Maxim', 'Vladimir', 'Alex'], 'surname': ['Dolgih', 'Petrova', 'Ivanov', 'Sidorov', 'Manuylov'], 'age': ['23', '44', '55', '2', '5'], 'have_pets': ['True', 'True', 'True', 'True', 'True']} 

таблица создана
pickle
{'name': ['Yana', 'Maria', 'Maxim', 'Vladimir', 'Peter'], 'surname': ['Dolgih', 'Petrova', 'Ivanov', 'Sidorov', 'Gavrilov'], 'age': [20, 30, 22, 52, 15], 'have_pets': [True, True, True, True, True], 'reg_date': [datetime.datetime(2024, 11, 23, 12, 30), datetime.datetime(2024, 11, 24, 12, 40), datetime.datetime(2024, 1

In [5]:
def get_rows_by_number(tab, start=1, stop='none', copy_table=False):
    """
    Extracts rows from a table (dictionary) based on row numbers.
    This function extracts a subset of rows from a table represented as a dictionary,
    where keys are column names and values are lists of column data.
    
    Args:
        tab: The input table (dict). Keys are column names, values are lists of data.
        start: The starting row number (int, 1-based index). Defaults to 1.
        stop: The ending row number (int, 1-based index). Defaults to 'none', which means
              all rows from `start` to the end will be extracted. 
        copy_table: If True, returns a deep copy of the extracted rows as a list of lists;
                    otherwise, returns the extracted rows as a dictionary with column
                    names as keys. Defaults to False.
    
    
    Returns:
        A dictionary where keys are column names from the input
        table and values are lists containing the extracted rows.
        Returns an error message 'ошибка ввода!' if any error occurs during processing.
    
    
    Raises:
        All exceptions are caught and a generic error message is returned.
    
    Example Usage:
      >>> tab = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
      >>> get_rows_by_number(tab, start=2, stop=3)
      {'A': [2], 'B': [5], 'C': [8]}
    
    """
    try:
        key=list(tab.keys())
        if stop == 'none':
            stop = len(tab.keys()) + 1
        val = list(tab.values())
        newtab = []
        for el in range(start - 1, stop):
            line = []
            for i in range(len(tab.keys())):
                line.append(val[i][el])
            newtab.append(line)
        
        res = []
        for col in range(len(key)):
            lst1 = []
            for ln in range(len(newtab)):
                lst1.append(newtab[ln][col])
            res.append(lst1)
        dic = {key[num]: res[num] for num in range(len(key))}
        if copy_table == True:
            return copy.deepcopy(newtab)
        return dic
    except:return 'ошибка ввода!'

In [6]:
print(get_rows_by_number(tab, 1, 3))

{'name': ['Yana', 'Maria', 'Maxim'], 'surname': ['Dolgih', 'Petrova', 'Ivanov'], 'age': [20, 30, 22], 'have_pets': [True, True, True], 'reg_date': [datetime.datetime(2024, 11, 23, 12, 30), datetime.datetime(2024, 11, 24, 12, 40), datetime.datetime(2024, 11, 25, 12, 32)]}


In [7]:
def get_rows_by_index(tab, vals1, copy_table=False):
    """
    Extracts rows from a table (dictionary) based on row indices.
    This function extracts specified rows from a table represented as a dictionary, where
    keys are column names and values are lists of column data.  The row indices are
    0-based.
    
    Args:
        tab: The input table (dict). Keys are column names; values are lists of data.
        vals1: A list of row indices (int) to extract (1-based index).
        copy_table: If True, returns a deep copy of the extracted rows as a list of lists;
                    otherwise, returns the extracted rows as a dictionary with column
                    names as keys. Defaults to False.
    
    Returns:
        A dictionary where keys are column names from the input
        table and values are lists containing the extracted rows.  
        
    Raises:
        All exceptions are caught and a generic error message is returned. 

    Example Usage:
      >>> tab = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
      >>> get_rows_by_index(tab, [1, 3])
      {'A': [1, 3], 'B': [4, 6], 'C': [7, 9]}


    """
    try:
        vals = [val-1 for val in vals1]
        key = list(tab.keys())
        val = list(tab.values())
        newtab = []
        for el in set(vals):
            line = []
            for i in range(len(tab.keys())):
                line.append(val[i][el])
            newtab.append(line)
        res = []
        for col in range(len(key)):
            lst1 = []
            for ln in range(len(newtab)):
                lst1.append(newtab[ln][col])
            res.append(lst1)
        dic = {key[num]: res[num] for num in range(len(key))}
        if copy_table == True:
            return copy.deepcopy(newtab)
        return dic
    except:return 'ошибка ввода!'


In [8]:
print(get_rows_by_index(tab, [2, 5]))

{'name': ['Maria', 'Peter'], 'surname': ['Petrova', 'Gavrilov'], 'age': [30, 15], 'have_pets': [True, True], 'reg_date': [datetime.datetime(2024, 11, 24, 12, 40), datetime.datetime(2024, 11, 27, 12, 20)]}


In [9]:
def set_column_types(tab, typec, by_number = True,col = 0):
    """
    Sets the data type of a specified column in a table (dictionary).
    
    This function modifies a table (represented as a dictionary) by converting the
    elements of a specified column to a new data type.
    
    Args:
        tab: The input table (dict). Keys are column names, values are lists of data.
        typec: The target data type (e.g., int, float, str).  A function that converts
               values to the desired type.  Must be able to handle potential errors
               during type conversion.
        by_number: If True, the column is specified by its 1-based index (`col`);
                   otherwise, it's specified by its name (`col`). Defaults to True.
        col: The column to modify. If `by_number` is True, this is the 1-based index of
             the column; otherwise, it's the column name (str).
    
    Returns:
        'тип задан' if the column type was changed, or an error message 'ошибка ввода!'
        if an error occurred.
    
    Raises:
        All exceptions are caught and a generic error message is returned.
    
    Example Usage:
      >>> tab = {'A': ['1', '2', '3'], 'B': ['4', '5', '6']}
      >>> set_column_types(tab, int, by_number=True, col=1)
      тип задан
 
    """
    try:
        if by_number:
            tab[list(tab.keys())[col - 1]] = list(map(typec, tab[list(tab.keys())[col - 1]]))
        else:
            tab[col] = list(map(typec, tab[col]))
        return 'тип задан'
    except:return 'ошибка ввода!'


In [10]:

print(set_column_types(tab, int, True,3))
print(set_column_types(tab, bool, False,'have_pets'))

тип задан
тип задан


In [11]:
def get_column_types(tab, by_number=True,col=0):
    """
    Gets the data type of the first element in a specified column.
    This function retrieves the data type of the first element in a specified column
    of a table represented as a dictionary.
    
    Args:
        tab: The input table (dict). Keys are column names, values are lists of data.
        by_number: If True, the column is specified by its 1-based index (`col`);
                   otherwise, it's specified by its name (`col`). Defaults to True.
        col: The column to check. If `by_number` is True, this is the 1-based index;
             otherwise, it's the column name (str).
    
    Returns:
        The data type of the first element in the specified column. Returns an error
        message 'ошибка ввода!' if an error occurs during processing.
    
    Raises:
        All exceptions are caught and a generic error message is returned.

    Example Usage:
      >>> tab = {'A': [1, 2, 3], 'B': ['4', '5', '6']}
      >>> get_column_types(tab, by_number=True, col=1)
      <class 'int'>
    
    """
    try:
        if by_number:
            return type(tab[list(tab.keys())[col - 1]][0])
        else:
            return type(tab[col][0])
    except:return 'ошибка ввода!'

In [12]:
print(get_column_types(tab, True,3))
print(get_column_types(tab, False,'have_pets'))

print(get_column_types(tab,True,1))
print(get_column_types(tab,True,3))
print(get_column_types(tab,True,4))
print(get_column_types(tab,True,5))

<class 'int'>
<class 'bool'>
<class 'str'>
<class 'int'>
<class 'bool'>
<class 'datetime.datetime'>


In [13]:
def get_values(tab,col=0):
    """Retrieves data from a specified column in a table.

    Args:
        col:  The column identifier.  Can be:
              - An integer (1-based index):  If provided as an integer, it represents
                the column index (starting from 1).
              - A string: If provided as a string, it represents the column name.
              Defaults to 0 

    Returns:
        A list containing the values of the specified column.
        Returns the string 'ошибка ввода!' (input error) if an error occurs.

    Raises:
        All exceptions are caught and a generic error message is returned.

    Example Usage:
        >>> tab = {'col1': [1, 2, 3], 'col2': ['a', 'b', 'c']}
        >>> get_values('col1')
        [1, 2, 3]
    """
    try:
        if isinstance(col, int):
            if col <= 0:  
                raise IndexError("Неверный индекс")
            ans = tab[list(tab.keys())[col - 1]] 
        elif isinstance(col, str):
            ans = tab[col]
        else:
            raise TypeError("ошибка: тип аргумента")
        return ans
    except: return 'ошибка ввода!' 

In [14]:
print(get_values(tab,2))
print(get_values(tab,'name'))

['Dolgih', 'Petrova', 'Ivanov', 'Sidorov', 'Gavrilov']
['Yana', 'Maria', 'Maxim', 'Vladimir', 'Peter']


In [15]:
def get_value(tab,col=0, ln=0):
    """
    This function accesses a value from a tabular data structure represented as a
    dictionary, where keys are column identifiers and values are lists
    representing rows.  It handles both numeric and non-numeric column
    identifiers.
    
    Args:
    col: The column index or identifier.  If numeric, it represents the
         1-based index of the column.  Otherwise, it's treated as a direct key
         into the dictionary. Defaults to 0.
    ln: The 1-based index of the row. Defaults to 0.
    
    Returns:
    The value at the specified column and row.  Returns "ошибка ввода!" (error
    input) if an error occurs during access (e.g., invalid column or row index,
    missing key).
    
    Raises:
    All exceptions are caught and a generic error message is returned.
    
    Example Usage:
    >>>tab = {'A': [1, 2, 3], 'B': [4, 5, 6]}
    >>>print(get_value(col='A', ln=2))  
    Output: 2

  """
    try:
        if str(col).isnumeric():
            ans = tab[list(tab.keys())[col - 1]][ln - 1]
        else:
            ans = tab[col][ln-1]
        return ans
    except:return 'ошибка ввода!'


In [16]:
print(get_value(tab,2, 3))
print(get_value(tab,'name', 3))

Ivanov
Maxim


In [17]:
def set_values(tab,val, col=0):
    """
    Sets column values in a tabular dictionary.
    
    Args:
    tab: The input dictionary representing the tabular data.  Keys are column
         identifiers, and values are lists representing rows. This dictionary is
         modified in place.
    val: The new value to set for all rows in the specified column.
    col: The column identifier. If an integer, it's treated as a 1-based index;
         otherwise, it's treated as a dictionary key. Defaults to 0.
    
    Returns:
    "замена прошла" (replacement complete) if successful, "ошибка ввода!" (error
    input) otherwise.
    
    Raises:
    All exceptions are caught and a generic error message is returned.
    
    Examples:
    >>> tab = {'A': [1, 2, 3], 'B': [4, 5, 6]}
    >>> set_values(tab, 0, col='A')
    'замена прошла'
    """
    try:
        if str(col).isnumeric():
            tab[list(tab.keys())[col - 1]] = val
        else:
            tab[col] = val
        return 'замена прошла'
    except:return 'ошибка ввода!'

In [18]:
print(set_values(tab,[23, 44, 55, 2, 5], 3))
print(get_values(tab,3))
print(set_values(tab,[True, True, True, True, True], 'have_pets'))
print(get_values(tab,'have_pets'))

замена прошла
[23, 44, 55, 2, 5]
замена прошла
[True, True, True, True, True]


In [19]:
def set_value(tab,val=0, col=0, ln=0):
    """
    Sets a single value in a dictionary.

    Args:
    tab: The input dictionary representing the tabular data. Keys are column
     identifiers, and values are lists representing rows. This dictionary is
     modified in place.
    val: The value to set in the specified cell. Defaults to 0.
    col: The column identifier. If an integer, it's treated as a 1-based index;
     otherwise, it's treated as a dictionary key. Defaults to 0.
    ln: The 1-based row index. Defaults to 0.
    
    Returns:
    "замена прошла" (replacement complete) if successful, "ошибка ввода!" (error
    input) otherwise.
    
    Raises:
    All exceptions are caught and a generic error message is returned.
    
    Examples:
    >>> tab = {'A': [1, 2, 3], 'B': [4, 5, 6]}
    >>> set_value(tab, 10, col='A', ln=1)
    'замена прошла'

    """
    try:
        if str(col).isnumeric():
            tab[list(tab.keys())[col - 1]][ln - 1] = val
        else:
            tab[col][ln - 1] = val
        return 'замена прошла'
    except:return 'ошибка ввода!'


In [20]:
print(set_value(tab,'Manuylov', 2, 5))
print(get_values(tab,2))
print(set_value(tab,'Alex', 'name', 5))
print(get_values(tab,'name'))

замена прошла
['Dolgih', 'Petrova', 'Ivanov', 'Sidorov', 'Manuylov']
замена прошла
['Yana', 'Maria', 'Maxim', 'Vladimir', 'Alex']


In [21]:
def print_table(tab):
    """
    Prints a dictionary to the console.

    Args:
    tab: A dictionary where keys represent column headers and values are lists
     representing rows of data.  All inner lists (rows) must have the same
     length.
    
    Returns:
    An empty string if the table is printed successfully, "ошибка ввода!" (error
    input) otherwise.
    
    Raises:
    All exceptions are caught and a generic error message is returned.
    
    Examples:
    >>> tab = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
    >>> print_table(tab)
    A B C
    1 4 7
    2 5 8
    3 6 9
    """
    try:
        key = tab.keys()
        val = list(tab.values())
        print(*key)
        for ln in range(len(key)):
            line = []
            for cl in range(len(val)):
                line.append(str(val[cl][ln]))
            print(*line)
        return ''
    except:return 'ошибка ввода!'

In [22]:
print(print_table(tab))

name surname age have_pets reg_date
Yana Dolgih 23 True 2024-11-23 12:30:00
Maria Petrova 44 True 2024-11-24 12:40:00
Maxim Ivanov 55 True 2024-11-25 12:32:00
Vladimir Sidorov 2 True 2024-11-26 12:50:00
Alex Manuylov 5 True 2024-11-27 12:20:00

